# Importing Packages and Libraries:
For the training of Word2Vec,i will be using Gensim.It is a widely used library for NLP works.

Gensim is an open-source library for unsupervised topic modeling and natural language processing.

In [3]:
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
from unidecode import unidecode
import os
import glob
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from time import time 
from collections import defaultdict

In [4]:
#turn on internet option in kernel
!pip install bangla-stemmer

In [5]:
from bangla_stemmer.stemmer import stemmer

In [6]:

def text_to_word_list(text):
    text = text.split()
    return text

def replace_strings(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\u00C0-\u017F"          #latin
                           u"\u2000-\u206F"          #generalPunctuations
                               
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    #latin_pattern=re.compile('[A-Za-z\u00C0-\u00D6\u00D8-\u00f6\u00f8-\u00ff\s]*',)
    
    text=emoji_pattern.sub(r'', text)
    text=english_pattern.sub(r'', text)

    return text

def stopwordRemoval(text):    
    x=str(text)
    l=x.split()

    stm=[elem for elem in l if elem not in stop]
    
    out=' '.join(stm)
    
    return str(out)

def remove_punctuations(my_str):
    # define punctuation
    punctuations = '''````£|¢|Ñ+-*/=EROero৳০১২৩৪৫৬৭৮৯012–34567•89।!()-[]{};:'"“\’,<>./?@#$%^&*_~‘—॥”‰⚽️✌�￰৷￰'''
    
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct



def joining(text):
    out=' '.join(text)
    return out

def preprocessing(text):
    out=remove_punctuations(replace_strings(text))
    return out


def Stemming(text):
    
    x=str(text)
    l=x.split()

    stmr = stemmer.BanglaStemmer()
    stm = stmr.stem(l)

    out=' '.join(stm)
    
    return str(out)


### Datasets are loaded here:
The datasets are the corpus and the stopwordslist.


In [7]:
df =pd.read_csv('main_dataset_v3.csv')
data1 =pd.read_excel('stopwords_bangla.xlsx')
stop = data1['words'].tolist()

I have only taken the sentences which are only in limited range if 100 to 1.

# Preprocessing

As the first step of preprocessing,I have cleaned the datase from unnecessary things with my predefined function Preprocessing().

In [9]:
df['Sentence'] = df.Sentence.apply(lambda x: preprocessing(str(x)))

Here i have applied Stemming.

In [10]:
#df['Sentence'] = df.Sentence.apply(lambda x: Stemming(str(x)))

Stopwords removal function is applied here.

In [11]:
df['Sentence'] = df.Sentence.apply(lambda x: stopwordRemoval(str(x)))

A histogram is plotted here to visualize the dataset characteristics.

Preprocessed sentences are converted into wordlist as the word2vec requiers word list for each sentence.

In [13]:
df['Sentence'] = df.Sentence.apply(lambda x: text_to_word_list(str(x)))
word2vecinput = [row for row in df.Sentence]

# Training Model

In [14]:

model = Word2Vec(word2vecinput,window=20,min_count=5,sg=0,negative=3,workers=multiprocessing.cpu_count()-1)


In [15]:
model.save("word2vec_model.model")

# Result

In [16]:
from gensim.models import Word2Vec

# Load the Word2Vec model from the file
model = Word2Vec.load("word2vec_model.model")

here we can see that our trained Word2vec embeddings is well trained.As we can see from examples,it can predict the similar words pretty good.

In [17]:
print(model.wv.most_similar("মা", topn=5))

[('মাকে', 0.8262583017349243), ('বাবা', 0.823793351650238), ('মাবাবা', 0.8149698376655579), ('স্বামীর', 0.8085119128227234), ('মেয়েকে', 0.8040454387664795)]


In [16]:
!pip install flask

In [18]:
print(model.wv.most_similar("খুলনা", topn=5))

[('বরিশাল', 0.9098139405250549), ('সিলেট', 0.8930426239967346), ('রাজশাহী', 0.863511323928833), ('রংপুর', 0.8353511095046997), ('রাজশাহীর', 0.8289179801940918)]


In [ ]:
from flask import Flask, request, render_template
from gensim.models import Word2Vec



app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/get_similarity', methods=['POST'])
def get_similarity():
    if request.method == 'POST':
        word = request.form['word']
        similar_words = model.wv.most_similar(word, topn=5)
        return render_template('result.html', word=word, similar_words=similar_words)

if __name__ == '__main__':
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [19]:
print(model.wv.most_similar("রোজা", topn=5))

[('সিয়াম', 0.8883941769599915), ('রোজা', 0.883510410785675), ('উপবাস', 0.8675622940063477), ('মাহে', 0.82308030128479), ('সংযম', 0.7846404314041138)]


In [ ]:
print(render_template('index.html'))

In [20]:
print(model.wv.most_similar("অপরাধ", topn=5))

[('অপরাধের', 0.8259578347206116), ('অপরাধে', 0.7584247589111328), ('অপরাধগুলোর', 0.7536808848381042), ('ফৌজদারি', 0.7304179072380066), ('অপরাধীদের', 0.7231983542442322)]


Here the function compute cosine similarity between two words.We can see that words are very similar as the value.

